In [1]:
# Import our dependencies
import pandas as pd
import matplotlib as plt
import sklearn as skl
import tensorflow as tf

In [30]:
# Read in our charity data
charity_df = pd.read_csv("charity_data.csv")

In [3]:
charity_df.tail(50)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
34249,990297069,PHAC NO 1 INC,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,3999445,0
34250,990298651,SUTTER HEALTH PACIFIC,T3,Independent,C1000,Heathcare,Trust,1,10M-50M,N,14241629,1
34251,990299264,HEALTHY MOTHERS HEALTHY BABIES COALITION OF HA...,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,299684,0
34252,990300906,MAUI COUNTY COMMUNITY TELEVISION INC,T3,Independent,C1000,Preservation,Trust,1,1M-5M,N,3822376,0
34253,990303677,KEKAHU FOUNDATION,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,267157,0
34254,990303731,KANEOHE BAY LODGE NO 2,T8,Independent,C1000,ProductDev,Association,1,0,N,5000,1
34255,990304187,IMPACT TEAMS INTERNATIONAL,T3,Independent,C1000,Preservation,Trust,1,0,N,5000,1
34256,990304613,NATIONAL ASSOCIATION OF CHARITABLE GIFT PLANNERS,T3,CompanySponsored,C2000,Preservation,Trust,1,0,N,5000,1
34257,990306548,H R A INC,T3,Independent,C1000,Preservation,Trust,1,0,N,5000,1
34258,990310248,THE UNITED STATES PONY CLUBS INC,T3,CompanySponsored,C1200,Preservation,Association,1,0,N,5000,1


In [31]:
# characterize the data
# EIN and NAME - remove from dataset

# APPLICATION_TYPE t3 vs other
apptype_counts = charity_df.APPLICATION_TYPE.value_counts(dropna=False)
print(apptype_counts.head(25))

# Determine which values to replace
replace_avalue = list(apptype_counts[apptype_counts != 27037].index)

# Replace in DataFrame
for avalue in replace_avalue:
    charity_df.APPLICATION_TYPE = charity_df.APPLICATION_TYPE.replace(avalue,"OtherThanT3")


# Check to make sure binning was successful
charity_df.APPLICATION_TYPE.value_counts(dropna=False)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


T3             27037
OtherThanT3     7262
Name: APPLICATION_TYPE, dtype: int64

In [33]:
# AFFILIATION 2 classes
AFFILIATION_counts = charity_df.AFFILIATION.value_counts(dropna=False)
print(AFFILIATION_counts.head(25))

# Determine which values to replace
replace_AFFILIATION = list(AFFILIATION_counts[(AFFILIATION_counts < 100) | (AFFILIATION_counts > 16000)].index)

# Replace in DataFrame
for country in replace_AFFILIATION:
    charity_df.AFFILIATION = charity_df.AFFILIATION.replace(country,"NotCompanySponsored")


# Check to make sure binning was successful
charity_df.AFFILIATION.value_counts(dropna=False)

NotCompanySponsored    18594
CompanySponsored       15705
Name: AFFILIATION, dtype: int64


NotCompanySponsored    18594
CompanySponsored       15705
Name: AFFILIATION, dtype: int64

In [34]:
# CLASSIFICATION C1000 vs other
CLASSIFICATION_counts = charity_df.CLASSIFICATION.value_counts(dropna=False)
print(CLASSIFICATION_counts.head(25))

# Determine which values to replace
replace_CLASSIFICATION = list(CLASSIFICATION_counts[CLASSIFICATION_counts < 2000].index)

# Replace in DataFrame
for country in replace_CLASSIFICATION:
    charity_df.CLASSIFICATION = charity_df.CLASSIFICATION.replace(country,"Other")


# Check to make sure binning was successful
charity_df.CLASSIFICATION.value_counts(dropna=False)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
Name: CLASSIFICATION, dtype: int64


C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [35]:
# USE_CASE 2 classes
USE_CASE_counts = charity_df.USE_CASE.value_counts(dropna=False)
print(USE_CASE_counts.head(25))

# Determine which values to replace
replace_USE_CASE = list(USE_CASE_counts[(USE_CASE_counts < 1000) | (USE_CASE_counts > 10000)].index)

# Replace in DataFrame
for country in replace_USE_CASE:
    charity_df.USE_CASE = charity_df.USE_CASE.replace(country,"OtherThanProductDev")


# Check to make sure binning was successful
charity_df.USE_CASE.value_counts(dropna=False)

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64


OtherThanProductDev    28628
ProductDev              5671
Name: USE_CASE, dtype: int64

In [36]:
# ORGANIZATION 2 classes
ORGANIZATION_counts = charity_df.ORGANIZATION.value_counts(dropna=False)
print(ORGANIZATION_counts.head(25))

# Determine which values to replace
replace_ORGANIZATION = list(ORGANIZATION_counts[(ORGANIZATION_counts < 1000) | (ORGANIZATION_counts > 15000)].index)

# Replace in DataFrame
for country in replace_ORGANIZATION:
    charity_df.ORGANIZATION = charity_df.ORGANIZATION.replace(country,"OtherThanAssociation")


# Check to make sure binning was successful
charity_df.ORGANIZATION.value_counts(dropna=False)

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64


OtherThanAssociation    24044
Association             10255
Name: ORGANIZATION, dtype: int64

In [9]:
# # INCOME_AMT - start with none vs any; ordinal classes: 0; 1-9999, 10000-24999, 25000-99999, 100000-499999

# INCOME_AMT_counts = charity_df.INCOME_AMT.value_counts(dropna=False)
# print(INCOME_AMT_counts.head(25))

# # Determine which values to replace
# replace_INCOME_AMT = list(INCOME_AMT_counts[INCOME_AMT_counts < 5000].index)

# # Replace in DataFrame
# for country in replace_INCOME_AMT:
#     charity_df.INCOME_AMT = charity_df.INCOME_AMT.replace(country,"AnyIncome")


# # Check to make sure binning was successful
# charity_df.INCOME_AMT.value_counts()



In [37]:
# STATUS -remove
STATUS_counts = charity_df.STATUS.value_counts(dropna=False)
print(STATUS_counts.head(25))


1    34294
0        5
Name: STATUS, dtype: int64


In [38]:
# charity_df['incomeCategoryRank']
charity_df.loc[charity_df['INCOME_AMT'] == '0', 'incomeGroupRank'] = 0

charity_df.loc[charity_df['INCOME_AMT'] == '1-9999', 'incomeGroupRank'] = 1
charity_df.loc[charity_df['INCOME_AMT'] == '10000-24999', 'incomeGroupRank'] = 2
charity_df.loc[charity_df['INCOME_AMT'] == '25000-99999', 'incomeGroupRank'] = 3
charity_df.loc[charity_df['INCOME_AMT'] == '100000-499999', 'incomeGroupRank'] = 4
charity_df.loc[charity_df['INCOME_AMT'] == '1M-5M', 'incomeGroupRank'] = 5
charity_df.loc[charity_df['INCOME_AMT'] == '5M-10M', 'incomeGroupRank'] = 6
charity_df.loc[charity_df['INCOME_AMT'] == '10M-50M', 'incomeGroupRank'] = 7
charity_df.loc[charity_df['INCOME_AMT'] == '50M+', 'incomeGroupRank'] = 8
          
            
charity_df.head(20)


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,incomeGroupRank
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,OtherThanT3,NotCompanySponsored,C1000,ProductDev,Association,1,0,N,5000,1,0.0
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,NotCompanySponsored,C2000,OtherThanProductDev,OtherThanAssociation,1,1-9999,N,108590,1,1.0
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,OtherThanT3,CompanySponsored,Other,ProductDev,Association,1,0,N,5000,0,0.0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,OtherThanProductDev,OtherThanAssociation,1,10000-24999,N,6692,1,2.0
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,NotCompanySponsored,C1000,OtherThanProductDev,OtherThanAssociation,1,100000-499999,N,142590,1,4.0
5,10556855,MINORITY ORGAN & TISSUE TRANSPLANT & EDUCATION...,T3,NotCompanySponsored,C1200,OtherThanProductDev,OtherThanAssociation,1,0,N,5000,1,0.0
6,10558440,FRIENDS OF ARTS COUNCIL OF GREATER DENHAM SPRI...,T3,NotCompanySponsored,C1000,OtherThanProductDev,OtherThanAssociation,1,100000-499999,N,31452,1,4.0
7,10566033,ISRAEL EMERGENCY ALLIANCE,T3,NotCompanySponsored,C2000,OtherThanProductDev,OtherThanAssociation,1,10M-50M,N,7508025,1,7.0
8,10570430,ARAMCO BRATS INC,OtherThanT3,NotCompanySponsored,C1000,ProductDev,OtherThanAssociation,1,1-9999,N,94389,1,1.0
9,10571689,INTERNATIONAL ASSOCIATION OF FIRE FIGHTERS,OtherThanT3,CompanySponsored,Other,ProductDev,Association,1,0,N,5000,0,0.0


In [39]:
# charity_df['incomeCategoryRank']
charity_df.loc[charity_df['INCOME_AMT'] == '0', 'incomeGroupMed'] = 0

charity_df.loc[charity_df['INCOME_AMT'] == '1-9999', 'incomeGroupMed'] = 5000
charity_df.loc[charity_df['INCOME_AMT'] == '10000-24999', 'incomeGroupMed'] = 17500
charity_df.loc[charity_df['INCOME_AMT'] == '25000-99999', 'incomeGroupMed'] = 62500
charity_df.loc[charity_df['INCOME_AMT'] == '100000-499999', 'incomeGroupMed'] = 300000
charity_df.loc[charity_df['INCOME_AMT'] == '1M-5M', 'incomeGroupMed'] = 3000000
charity_df.loc[charity_df['INCOME_AMT'] == '5M-10M', 'incomeGroupMed'] = 7500000
charity_df.loc[charity_df['INCOME_AMT'] == '10M-50M', 'incomeGroupMed'] = 30000000
charity_df.loc[charity_df['INCOME_AMT'] == '50M+', 'incomeGroupMed'] = 50000000
          
            
charity_df.head(20)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,incomeGroupRank,incomeGroupMed
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,OtherThanT3,NotCompanySponsored,C1000,ProductDev,Association,1,0,N,5000,1,0.0,0.0
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,NotCompanySponsored,C2000,OtherThanProductDev,OtherThanAssociation,1,1-9999,N,108590,1,1.0,5000.0
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,OtherThanT3,CompanySponsored,Other,ProductDev,Association,1,0,N,5000,0,0.0,0.0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,OtherThanProductDev,OtherThanAssociation,1,10000-24999,N,6692,1,2.0,17500.0
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,NotCompanySponsored,C1000,OtherThanProductDev,OtherThanAssociation,1,100000-499999,N,142590,1,4.0,300000.0
5,10556855,MINORITY ORGAN & TISSUE TRANSPLANT & EDUCATION...,T3,NotCompanySponsored,C1200,OtherThanProductDev,OtherThanAssociation,1,0,N,5000,1,0.0,0.0
6,10558440,FRIENDS OF ARTS COUNCIL OF GREATER DENHAM SPRI...,T3,NotCompanySponsored,C1000,OtherThanProductDev,OtherThanAssociation,1,100000-499999,N,31452,1,4.0,300000.0
7,10566033,ISRAEL EMERGENCY ALLIANCE,T3,NotCompanySponsored,C2000,OtherThanProductDev,OtherThanAssociation,1,10M-50M,N,7508025,1,7.0,30000000.0
8,10570430,ARAMCO BRATS INC,OtherThanT3,NotCompanySponsored,C1000,ProductDev,OtherThanAssociation,1,1-9999,N,94389,1,1.0,5000.0
9,10571689,INTERNATIONAL ASSOCIATION OF FIRE FIGHTERS,OtherThanT3,CompanySponsored,Other,ProductDev,Association,1,0,N,5000,0,0.0,0.0


In [40]:
# SPECIAL_CONSIDERATIONS - drop
SPECIAL_CONSIDERATIONS_counts = charity_df.SPECIAL_CONSIDERATIONS.value_counts(dropna=False)
print(SPECIAL_CONSIDERATIONS_counts.head(25))

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64


In [41]:
# ASK_AMT 
charity_df['askAmount'] = charity_df['ASK_AMT'].astype('float64')

In [42]:
charity_df['APPLICATION_TYPE'] = charity_df['APPLICATION_TYPE'].astype('str')
charity_df['AFFILIATION'] = charity_df['AFFILIATION'].astype('str')
charity_df['CLASSIFICATION'] = charity_df['CLASSIFICATION'].astype('str')
charity_df['USE_CASE'] = charity_df['USE_CASE'].astype('str')
charity_df['ORGANIZATION'] = charity_df['ORGANIZATION'].astype('str')

# Visualize the value counts
print(charity_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   EIN                     34299 non-null  int64  
 1   NAME                    34299 non-null  object 
 2   APPLICATION_TYPE        34299 non-null  object 
 3   AFFILIATION             34299 non-null  object 
 4   CLASSIFICATION          34299 non-null  object 
 5   USE_CASE                34299 non-null  object 
 6   ORGANIZATION            34299 non-null  object 
 7   STATUS                  34299 non-null  int64  
 8   INCOME_AMT              34299 non-null  object 
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object 
 10  ASK_AMT                 34299 non-null  int64  
 11  IS_SUCCESSFUL           34299 non-null  int64  
 12  incomeGroupRank         34299 non-null  float64
 13  incomeGroupMed          34299 non-null  float64
 14  askAmount               34299 non-null

In [43]:
# IS_SUCCESSFUL yes vs no
IS_SUCCESSFUL_counts = charity_df.IS_SUCCESSFUL.value_counts(dropna=False)
print(IS_SUCCESSFUL_counts.head(25))

1    18261
0    16038
Name: IS_SUCCESSFUL, dtype: int64


In [44]:
# Name
name_counts = charity_df.NAME.value_counts(dropna=False)
print(name_counts.head(60))

PARENT BOOSTER USA INC                                                1260
TOPS CLUB INC                                                          765
UNITED STATES BOWLING CONGRESS INC                                     700
WASHINGTON STATE UNIVERSITY                                            492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                        408
PTA TEXAS CONGRESS                                                     368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                          331
ALPHA PHI SIGMA                                                        313
TOASTMASTERS INTERNATIONAL                                             293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                      287
LITTLE LEAGUE BASEBALL INC                                             277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                               266
MOMS CLUB                                                              210
INTERNATIONAL ASSOCIATION

In [18]:
# regular expressions to find matches
# vets_list = ['VETERANS', 'VFW', 'ARMY', 'NAVY', 'MARINE']
# kids_list = ['PTA', 'ISD', 'PARENT', '4-H']
# school_list = ['COLLEGE', 'UNIVERSITY', 'ALPHA', 'BETA','LAMBDA', 'THETA', 'DELTA','PHI', 'SIGMA', 'KAPPA', 'GAMMA', 'THETA', 'UPSILON' ]
# serv_list = ['HUMANITY', 'SERVICE', 'SOROPTIMIST', 'LIONS', 'ROTARY']

# orgtype = charity_df['NAME']

# # Determine which values to replace
# s1_NAME = list(charity_df['NAME'].str.contains('PTA').index)
# for vet in vets_list:
#     list(charity_df['NAME'].str.contains(vet).index)

# # Replace in DataFrame
# for kid in s1_NAME:
#     charity_df.NAME = "Veterans"
 
    
    
# # Check to make sure binning was successful
# print(charity_df.NAME.value_counts(dropna=False))

Veterans    34299
Name: NAME, dtype: int64


In [45]:
# Check to make sure binning was successful
print(charity_df.NAME.value_counts(dropna=False).head(50))

PARENT BOOSTER USA INC                                                1260
TOPS CLUB INC                                                          765
UNITED STATES BOWLING CONGRESS INC                                     700
WASHINGTON STATE UNIVERSITY                                            492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                        408
PTA TEXAS CONGRESS                                                     368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                          331
ALPHA PHI SIGMA                                                        313
TOASTMASTERS INTERNATIONAL                                             293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                      287
LITTLE LEAGUE BASEBALL INC                                             277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                               266
MOMS CLUB                                                              210
INTERNATIONAL ASSOCIATION

In [46]:
# alumni, veterans, civic organizations, 
veterans_list = ['DEPARTMENT OF OREGON LADIES AUXILIARY TO THE VFW OF THE U S', 'MARINE CORPS LEAGUE', 'MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS','KNIGHTS OF COLUMBUS', 'TENNESSEE ORDER OF THE EASTERN STAR', 'VETERANS OF FOREIGN WARS OF THE UNITED STATES AUXILIARY', 'AIR FORCE ASSOCIATION', 'DEMOLAY INTERNATIONAL', 'DISABLED AMERICAN VETERANS', 'VFW AUXILIARY DEPARTMENT OF OKLAHOMA INC', 'DEPARTMENT OF KANSAS LADIES AUXILIARY TO THE VFW OF THE US INC', 'SONS OF CONFEDERATE VETERANS INC', 'MARINE CORPS LEAGUE', 'DEPARTMENT OF OREGON LADIES AUXILIARY TO THE VFW OF THE U S']
pta_list = ['POP WARNER LITTLE SCHOLARS INC', 'CLEAR CREEK ISD GROUPS', 'PARENT BOOSTER USA INC', 'PTA TEXAS CONGRESS', 'LITTLE LEAGUE BASEBALL INC', 'MOMS CLUB', 'PTA UTAH CONGRESS', 'MONTANA 4-H FOUNDATION INC', 'CLEARCREEK I S D SUPPORT GROUPS', 'CISD EDUCATIONAL SUPPORT GROUPS INC']
college_list = ['AMERICAN ASSOCIATION OF UNIVERSITY WOMEN INC', 'UNIVERSITY OF FLORIDA','LAMBDA THETA ALPHA', 'WASHINGTON STATE UNIVERSITY', 'AMERICAN ASSOCIATION OF UNIVERSITY WOMEN', 'ALPHA PHI SIGMA', 'SIGMA BETA DELTA INC', 'HONOR SOCIETY OF PHI KAPPA PHI', 'UNIVERSITY OF WYOMING', 'GAMMA THETA UPSILON']
athleisure_list = ['MUSIC TEACHERS NATIONAL ASSOCIATION INC', 'INTERNATIONAL MOUNTAIN BICYCLING ASSOCIATION - COLORADO', 'AMERICAN YOUTH FOOTBALL INC', 'UNITED FEDERATION OF DOLL CLUBS INC', 'AMATEUR ATHLETIC UNION OF THE UNITED STATES', 'UNITED STATES BOWLING CONGRESS INC', 'AMATEUR ATHLETIC UNION OF THE UNITED STATES INC', 'TOASTMASTERS INTERNATIONAL', 'THE UNITED STATES PONY CLUBS INC', 'CALIFORNIA GARDEN CLUBS INC', 'SWEET ADELINES INTERNATIONAL']
service_list = ['ALABAMA FEDERATION OF WOMENS CLUBS', 'NATIONAL YOUNG MENS SERVICE LEAGUE', 'SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC', 'TOPS CLUB INC', 'INTERNATIONAL ASSOCIATION OF LIONS CLUBS', 'HABITAT FOR HUMANITY INTERNATIONAL INC', 'CIVITAN INTERNATIONAL', 'WASHINGTON STATE GRANGE', 'SERTOMA INC', 'SOCIETY OF SAINT VINCENT DE PAUL COUNCIL OF LOS ANGELES', 'ROTARY INTERNATIONAL', 'KLEIN SUPPORT GROUPS INC', 'SLOW FOOD USA INC']
unions_list = ['INTERNATIONAL ASSOCIATION OF SHEET METAL AIR RAIL & TRANSPORTATION', 'FARMERS EDUCATIONAL AND COOPERATIVE UNION OF AMERICA', 'WORKERS UNITED','INTERNATIONAL ASSOCIATION OF FIRE FIGHTERS']

# Replace in DataFrame
for club in veterans_list:
    charity_df.NAME = charity_df.NAME.replace(club,"Veterans")
    
for club in pta_list:
    charity_df.NAME = charity_df.NAME.replace(club,"PTA_and_Parents")

for club in college_list:
    charity_df.NAME = charity_df.NAME.replace(club,"College")
    
for club in athleisure_list:
    charity_df.NAME = charity_df.NAME.replace(club,"Athleisure")    
    
for club in service_list:
    charity_df.NAME = charity_df.NAME.replace(club,"Service")
    
for club in unions_list:
    charity_df.NAME = charity_df.NAME.replace(club,"Unions")

# Determine which values to replace
name_counts = charity_df.NAME.value_counts(dropna=False)
other_list = list(name_counts[(name_counts < 50)].index)

for club in other_list:
    charity_df.NAME = charity_df.NAME.replace(club,"Other")

group_counts = charity_df.NAME.value_counts(dropna=False)
print(group_counts.head(25))

Other              24145
PTA_and_Parents     2564
Service             2247
Athleisure          1929
College             1581
Veterans            1273
Unions               560
Name: NAME, dtype: int64


In [47]:
# drop EIN, STATUS, SPECIAL_CONSIDERATIONS, ASK_AMT
# df.drop(['B', 'C'], axis=1)
charity_df = charity_df.drop(['EIN', 'STATUS', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'INCOME_AMT'], axis=1)
charity_df.tail()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,IS_SUCCESSFUL,incomeGroupRank,incomeGroupMed,askAmount
34294,Other,OtherThanT3,NotCompanySponsored,C1000,ProductDev,Association,0,0.0,0.0,5000.0
34295,Service,OtherThanT3,CompanySponsored,Other,ProductDev,Association,0,0.0,0.0,5000.0
34296,Other,T3,CompanySponsored,C2000,OtherThanProductDev,Association,0,0.0,0.0,5000.0
34297,Other,OtherThanT3,NotCompanySponsored,Other,ProductDev,Association,1,0.0,0.0,5000.0
34298,Other,T3,NotCompanySponsored,C1000,OtherThanProductDev,OtherThanAssociation,0,5.0,3000000.0,36500179.0


In [48]:
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
# Use sklearn to split dataset
from sklearn.model_selection import train_test_split
# Create the OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [49]:
# Fit the encoder and produce encoded DataFrame
a_df = pd.DataFrame(enc.fit_transform(charity_df.APPLICATION_TYPE.values.reshape(-1,1)))
a_df.columns = enc.get_feature_names(['APPLICATION_TYPE'])
b_df = pd.DataFrame(enc.fit_transform(charity_df.AFFILIATION.values.reshape(-1,1)))
b_df.columns = enc.get_feature_names(['AFFILIATION'])
c_df = pd.DataFrame(enc.fit_transform(charity_df.CLASSIFICATION.values.reshape(-1,1)))
c_df.columns = enc.get_feature_names(['CLASSIFICATION'])
d_df = pd.DataFrame(enc.fit_transform(charity_df.USE_CASE.values.reshape(-1,1)))
d_df.columns = enc.get_feature_names(['USE_CASE'])
e_df = pd.DataFrame(enc.fit_transform(charity_df.ORGANIZATION.values.reshape(-1,1)))
e_df.columns = enc.get_feature_names(['ORGANIZATION'])
f_df = pd.DataFrame(enc.fit_transform(charity_df.NAME.values.reshape(-1,1)))
f_df.columns = enc.get_feature_names(['NAME'])

In [50]:
ab = a_df.merge(b_df,left_index=True, right_index=True, how='left')
abc = ab.merge(c_df,left_index=True, right_index=True, how='left')
abcd = abc.merge(d_df,left_index=True, right_index=True, how='left')
abcde = abcd.merge(e_df,left_index=True, right_index=True, how='left')
abcdef_df = abcde.merge(f_df,left_index=True, right_index=True, how='left')
abcdef_df

,APPLICATION_TYPE_OtherThanT3,APPLICATION_TYPE_T3,AFFILIATION_CompanySponsored,AFFILIATION_NotCompanySponsored,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_Other,USE_CASE_OtherThanProductDev,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_OtherThanAssociation,NAME_Athleisure,NAME_College,NAME_Other,NAME_PTA_and_Parents,NAME_Service,NAME_Unions,NAME_Veterans
0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34295,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
34296,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34297,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [51]:
cols = ['APPLICATION_TYPE_T3', 'AFFILIATION_CompanySponsored', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C2000', 'CLASSIFICATION_C1200', 'USE_CASE_ProductDev', 'ORGANIZATION_Association', 'NAME_Athleisure', 'NAME_College', 'NAME_Other', 'NAME_PTA_and_Parents', 'NAME_Service', 'NAME_Unions', 'NAME_Veterans']
abcde_df = pd.DataFrame(abcdef_df, columns=cols)
abcde_df

,APPLICATION_TYPE_T3,AFFILIATION_CompanySponsored,CLASSIFICATION_C1000,CLASSIFICATION_C2000,CLASSIFICATION_C1200,USE_CASE_ProductDev,ORGANIZATION_Association,NAME_Athleisure,NAME_College,NAME_Other,NAME_PTA_and_Parents,NAME_Service,NAME_Unions,NAME_Veterans
0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34295,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
34296,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34297,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [52]:
cont_df = charity_df[['incomeGroupMed', 'askAmount', "IS_SUCCESSFUL"]]

char_df = cont_df.merge(abcdef_df, left_index=True, right_index=True, how='left')

print(char_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   incomeGroupMed                     34299 non-null  float64
 1   askAmount                          34299 non-null  float64
 2   IS_SUCCESSFUL                      34299 non-null  int64  
 3   APPLICATION_TYPE_OtherThanT3       34299 non-null  float64
 4   APPLICATION_TYPE_T3                34299 non-null  float64
 5   AFFILIATION_CompanySponsored       34299 non-null  float64
 6   AFFILIATION_NotCompanySponsored    34299 non-null  float64
 7   CLASSIFICATION_C1000               34299 non-null  float64
 8   CLASSIFICATION_C1200               34299 non-null  float64
 9   CLASSIFICATION_C2000               34299 non-null  float64
 10  CLASSIFICATION_Other               34299 non-null  float64
 11  USE_CASE_OtherThanProductDev       34299 non-null  flo

In [53]:
numeric_features = ['incomeGroupMed', 'askAmount']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[('num', numeric_transformer, numeric_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
scaled_data = preprocessor.fit_transform(char_df)

In [54]:
keepcols = ['APPLICATION_TYPE_T3', 'AFFILIATION_CompanySponsored', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C2000', 'CLASSIFICATION_C1200', 'USE_CASE_ProductDev', 'ORGANIZATION_Association', "IS_SUCCESSFUL", 'NAME_Athleisure', 'NAME_College', 'NAME_Other', 'NAME_PTA_and_Parents', 'NAME_Service', 'NAME_Unions', 'NAME_Veterans']

scaled = pd.DataFrame(scaled_data, columns=['incomeGroupMed_x', 'askAmount_x'])
chars_df = char_df[keepcols]
presplit_df = scaled.merge(chars_df, left_index=True,right_index=True, how='left')
presplit_df

,incomeGroupMed_x,askAmount_x,APPLICATION_TYPE_T3,AFFILIATION_CompanySponsored,CLASSIFICATION_C1000,CLASSIFICATION_C2000,CLASSIFICATION_C1200,USE_CASE_ProductDev,ORGANIZATION_Association,IS_SUCCESSFUL,NAME_Athleisure,NAME_College,NAME_Other,NAME_PTA_and_Parents,NAME_Service,NAME_Unions,NAME_Veterans
0,-0.140430,-0.031725,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,-0.139206,-0.030536,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,-0.140430,-0.031725,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,-0.136143,-0.031706,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,-0.066939,-0.030146,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,-0.140430,-0.031725,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34295,-0.140430,-0.031725,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
34296,-0.140430,-0.031725,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34297,-0.140430,-0.031725,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [55]:
# Split our preprocessed data into our features and target arrays
y = presplit_df[["IS_SUCCESSFUL"]]
X = presplit_df.drop("IS_SUCCESSFUL", axis=1)
X

,incomeGroupMed_x,askAmount_x,APPLICATION_TYPE_T3,AFFILIATION_CompanySponsored,CLASSIFICATION_C1000,CLASSIFICATION_C2000,CLASSIFICATION_C1200,USE_CASE_ProductDev,ORGANIZATION_Association,NAME_Athleisure,NAME_College,NAME_Other,NAME_PTA_and_Parents,NAME_Service,NAME_Unions,NAME_Veterans
0,-0.140430,-0.031725,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,-0.139206,-0.030536,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,-0.140430,-0.031725,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,-0.136143,-0.031706,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,-0.066939,-0.030146,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,-0.140430,-0.031725,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34295,-0.140430,-0.031725,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
34296,-0.140430,-0.031725,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34297,-0.140430,-0.031725,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
X_train

,incomeGroupMed_x,askAmount_x,APPLICATION_TYPE_T3,AFFILIATION_CompanySponsored,CLASSIFICATION_C1000,CLASSIFICATION_C2000,CLASSIFICATION_C1200,USE_CASE_ProductDev,ORGANIZATION_Association,NAME_Athleisure,NAME_College,NAME_Other,NAME_PTA_and_Parents,NAME_Service,NAME_Unions,NAME_Veterans
30531,-0.140430,-0.031725,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
22775,-0.140430,-0.031725,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
19057,-0.140430,-0.031725,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
27536,-0.140430,-0.031725,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
21331,7.208692,0.095857,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20558,-0.066939,-0.031188,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
18547,-0.066939,-0.031149,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3476,-0.125120,-0.030596,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
22633,-0.140430,-0.031725,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [57]:
# Model A
# Define the model - deep neural net
number_input_features = 16
hidden_nodes_layer1 =  21
hidden_nodes_layer2 = 27

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train,y_train,epochs=75)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Model A Loss: {model_loss}, Accuracy: {model_accuracy}")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 21)                357       
_________________________________________________________________
dense_1 (Dense)              (None, 27)                594       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 28        
Total params: 979
Trainable params: 979
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/75
25724/25724 [==============================] - 1s 43us/sample - loss: 0.5618 - acc: 0.7073
Epoch 2/75
25724/25724 [==============================] - 1s 36us/sample -

25724/25724 [==============================] - 1s 37us/sample - loss: 0.5103 - acc: 0.7497
Epoch 66/75
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5102 - acc: 0.7497
Epoch 67/75
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5099 - acc: 0.7492
Epoch 68/75
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5099 - acc: 0.7513
Epoch 69/75
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5100 - acc: 0.7499
Epoch 70/75
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5100 - acc: 0.7493
Epoch 71/75
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5100 - acc: 0.7503
Epoch 72/75
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5100 - acc: 0.7490
Epoch 73/75
25724/25724 [==============================] - 1s 37us/sample - loss: 0.5098 - acc: 0.7506
Epoch 74/75
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5

In [58]:
# Model B
# Define the model - deep neural net
number_input_features = 16
hidden_nodes_layer1 =  30
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Model B Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 30)                510       
_________________________________________________________________
dense_4 (Dense)              (None, 30)                930       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 31        
Total params: 1,471
Trainable params: 1,471
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
25724/25724 [==============================] - 1s 39us/sample - loss: 0.5630 - acc: 0.7155
Epoch 2/100
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5270 - acc: 0.7419
Epoch 3/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5221 - acc: 0.7459
Epoch 4/100
25724/25724 [==============================] - 1s 38us/sample

25724/25724 [==============================] - 1s 36us/sample - loss: 0.5090 - acc: 0.7497
Epoch 73/100
25724/25724 [==============================] - 1s 38us/sample - loss: 0.5093 - acc: 0.7505
Epoch 74/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5085 - acc: 0.7511
Epoch 75/100
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5087 - acc: 0.7516
Epoch 76/100
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5085 - acc: 0.7500
Epoch 77/100
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5087 - acc: 0.7504
Epoch 78/100
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5085 - acc: 0.7510
Epoch 79/100
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5087 - acc: 0.7519
Epoch 80/100
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5085 - acc: 0.7512
Epoch 81/100
25724/25724 [==============================] - 1s 34us/sample - 

In [78]:
# Model C
# Define the model - deep neural net
number_input_features = 16
hidden_nodes_layer1 =  30
hidden_nodes_layer2 = 30
# hidden_nodes_layer3 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adamax", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Model C Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_68 (Dense)             (None, 30)                510       
_________________________________________________________________
dense_69 (Dense)             (None, 30)                930       
_________________________________________________________________
dense_70 (Dense)             (None, 1)                 31        
Total params: 1,471
Trainable params: 1,471
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5917 - acc: 0.6981
Epoch 2/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5484 - acc: 0.7216
Epoch 3/100
25724/25724 [==============================] - 1s 46us/sample - loss: 0.5358 - acc: 0.7338
Epoch 4/100
25724/25724 [==============================] - 1s 47us/sampl

25724/25724 [==============================] - 1s 39us/sample - loss: 0.5093 - acc: 0.7501
Epoch 72/100
25724/25724 [==============================] - 1s 39us/sample - loss: 0.5092 - acc: 0.7501
Epoch 73/100
25724/25724 [==============================] - 1s 39us/sample - loss: 0.5093 - acc: 0.7505
Epoch 74/100
25724/25724 [==============================] - 1s 39us/sample - loss: 0.5091 - acc: 0.7507
Epoch 75/100
25724/25724 [==============================] - 1s 39us/sample - loss: 0.5093 - acc: 0.7515
Epoch 76/100
25724/25724 [==============================] - 1s 39us/sample - loss: 0.5092 - acc: 0.7502
Epoch 77/100
25724/25724 [==============================] - 1s 42us/sample - loss: 0.5089 - acc: 0.7502
Epoch 78/100
25724/25724 [==============================] - 1s 39us/sample - loss: 0.5092 - acc: 0.7499
Epoch 79/100
25724/25724 [==============================] - 1s 39us/sample - loss: 0.5089 - acc: 0.7511
Epoch 80/100
25724/25724 [==============================] - 1s 39us/sample - 